In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kendalltau, pearsonr, spearmanr

def spearsonr_pval(x,y):
    return spearmanr(x,y)[1]

data_a = pd.read_csv("LVB-75DPI-metadata-feature-table.csv")

data_restore = pd.read_csv("LVB-mass-restored-9.csv")
data_restore_list = data_restore.filename.to_list()
data_restore_list.insert(0, "MouseID")

# this is metadata info, which is extra baylor information.
# careful for the range, [a,b] mean a <= X < b. Also python is 0 based index.
data_M = data_a.iloc[:, np.r_[0, 1:8]]
data_M = data_M.set_index('MouseID')
# this is metabolites, which is features.
data_F = data_a[data_restore_list]
data_F = data_F.set_index('MouseID')

# get the two part of column names: metadata colum name and features column name.
M_column_list = data_M.head(0)
print(M_column_list)
F_column_list = data_F.head(0)
print(F_column_list)

df_col_list = []
df_corr_list = []
df_pval_list = []

for m_col in M_column_list:
    for f_col in F_column_list:
        col_1 = data_M[m_col]
        col_2 = data_F[f_col]
        corr_col_1_2 = col_1.corr(col_2, method='spearman')
        corr_pval_col_1_2 = col_1.corr(col_2, method=spearsonr_pval)
        df_col_list.append(m_col + "-" + f_col)
        df_corr_list.append(corr_col_1_2)
        df_pval_list.append(corr_pval_col_1_2)

print(df_col_list)
print(df_corr_list)
print(df_pval_list)

import statsmodels.api

df_final = pd.DataFrame(list(zip(df_col_list, df_corr_list, df_pval_list)), columns=['col_name','corr', 'corr_pval'])

df_final['corr'].replace('', float("NaN"), inplace=True)
df_final.dropna(subset=['corr'], inplace=True)

dr_fdr_pval = statsmodels.stats.multitest.fdrcorrection(list(df_final["corr_pval"]), method='indep')
df_final['FDR_rejected'] = dr_fdr_pval[0]
df_final['FDR_pval'] = dr_fdr_pval[1]

df_final.to_csv('LVB-75DPI-group9-corr-pval.csv', index=False)

Empty DataFrame
Columns: [Heart.weight..Body.weight.ratio, Liver.weight..Body.weight.ratio, Ejection.Fraction, PR.Interval..s., QTc..s., P.Amplitude..V., CD3.CD8.IFNg.]
Index: []
Empty DataFrame
Columns: [X105.07_2.767, X249.112_2.768, X258.207_2.775, X267.123_2.767, X352.306_2.997, X369.17_2.767, X396.332_2.991, X430.223_2.769, X440.359_2.985, X484.385_2.982, X489.228_2.768, X599.241_2.765, X604.362_2.948, X790.38_2.768, X795.335_2.768]
Index: []
['Heart.weight..Body.weight.ratio-X105.07_2.767', 'Heart.weight..Body.weight.ratio-X249.112_2.768', 'Heart.weight..Body.weight.ratio-X258.207_2.775', 'Heart.weight..Body.weight.ratio-X267.123_2.767', 'Heart.weight..Body.weight.ratio-X352.306_2.997', 'Heart.weight..Body.weight.ratio-X369.17_2.767', 'Heart.weight..Body.weight.ratio-X396.332_2.991', 'Heart.weight..Body.weight.ratio-X430.223_2.769', 'Heart.weight..Body.weight.ratio-X440.359_2.985', 'Heart.weight..Body.weight.ratio-X484.385_2.982', 'Heart.weight..Body.weight.ratio-X489.228_2.768',